In [20]:
# Q3.2
# load data
import numpy as np
import pandas as pd
df = pd.read_csv("../data/data.csv")
df = df[["RANDID", "AGE", "SYSBP", "DIABP", "CURSMOKE", "BPMEDS", "CVD"]]
# df = df.dropna()

print("", df.shape[0], "rows")
observation_count = np.max(pd.value_counts(df["RANDID"]))


 11627 rows


In [21]:
# mean of continue/max for binary
df_mean = df.groupby(['RANDID']).aggregate(
    {'AGE': np.average, 'SYSBP': np.average, 'DIABP': np.average, 'CURSMOKE': np.max, 'BPMEDS': np.max, 'CVD': np.max})
df_mean.to_csv('../data/data_mean.csv', index=False)
print("", df.shape[0], "rows")

 11627 rows


In [22]:
# copy preprocess strategy
df_copy = pd.DataFrame()
id_count = df.groupby(['RANDID'], as_index=False)['RANDID'].aggregate({'COUNT':'count'})
# copy the last observation to fill in three observation for each subject
i = 0
for j in range(0,4434):
    b = int(id_count.loc[[j]]['COUNT'])
    if b == 1:
        df_copy = df_copy.append(df.loc[[i]], ignore_index=True)
        df_copy = df_copy.append(df.loc[[i]], ignore_index=True)
        df_copy = df_copy.append(df.loc[[i]], ignore_index=True)
        i = i+1
    elif b == 2:
        df_copy = df_copy.append(df.loc[[i]], ignore_index=True)
        df_copy = df_copy.append(df.loc[[i+1]], ignore_index=True)
        df_copy = df_copy.append(df.loc[[i+1]], ignore_index=True)
        i = i+2
    elif b == 3:
        df_copy = df_copy.append(df.loc[[i]], ignore_index=True)
        df_copy = df_copy.append(df.loc[[i+1]], ignore_index=True)
        df_copy = df_copy.append(df.loc[[i+2]], ignore_index=True)
        i = i+3

df_copy.to_csv('../data/data_copy.csv', index=False)
print("", df.shape[0], "rows")

 11627 rows


In [23]:
# miceone
# Use one vector to represent the three observation of a subject
df_mice = pd.DataFrame()
id_count = df.groupby(['RANDID'], as_index=False)['RANDID'].aggregate({'COUNT':'count'})

# modified the row from 3 observation to 1
def Three2One(row, age2, sysbp2,diabp2,cursmoke2
                 , age3, sysbp3,diabp3,cursmoke3):
    row["AGE2"] = age2
    row["AGE3"] = age3
    row["SYSBP2"] = sysbp2
    row["SYSBP3"] = sysbp3
    row["DIABP2"] = diabp2
    row["DIABP3"] = diabp3
    row["CURSMOKE2"] = cursmoke2
    row["CURSMOKE3"] = cursmoke3
    return row
i = 0
print(df_mice.shape)
for j in range(0,4434):
    b = int(id_count.loc[[j]]['COUNT'])
    # print(b)
    if b == 1:
        r = Three2One(df.loc[[i]], np.nan, np.nan, np.nan, np.nan,
                        np.nan, np.nan, np.nan, np.nan)
        df_mice = df_mice.append(r, ignore_index=True)
        i = i+1
    elif b == 2:
        r = Three2One(df.loc[[i]], df.loc[[i+1]].iloc[0,1], df.loc[[i+1]].iloc[0,2], df.loc[[i+1]].iloc[0,3], df.loc[[i+1]].iloc[0,4],
                        np.nan, np.nan, np.nan, np.nan)
        df_mice = df_mice.append(r, ignore_index=True)
        i = i+2
    elif b == 3:
        r = Three2One(df.loc[[i]], df.loc[[i+1]].iloc[0,1], df.loc[[i+1]].iloc[0,2], df.loc[[i+1]].iloc[0,3], df.loc[[i+1]].iloc[0,4],
                                   df.loc[[i+2]].iloc[0,1], df.loc[[i+2]].iloc[0,2], df.loc[[i+2]].iloc[0,3], df.loc[[i+2]].iloc[0,4])
        df_mice = df_mice.append(r, ignore_index=True)
        i = i+3

print(df_mice)

(0, 0)
       RANDID  AGE  SYSBP  DIABP  CURSMOKE  BPMEDS  CVD  AGE2  AGE3  SYSBP2  \
0        2448   39  106.0   70.0         0     0.0    1  52.0   NaN   121.0   
1        6238   46  121.0   81.0         0     0.0    0  52.0  58.0   105.0   
2        9428   48  127.5   80.0         1     0.0    0  54.0   NaN   141.0   
3       10552   61  150.0   95.0         1     0.0    1  67.0   NaN   183.0   
4       11252   46  130.0   84.0         1     0.0    0  51.0  58.0   109.0   
...       ...  ...    ...    ...       ...     ...  ...   ...   ...     ...   
4429  9990894   48  131.0   72.0         1     NaN    0  54.0  60.0   145.0   
4430  9993179   44  126.5   87.0         1     0.0    0  50.0  56.0   131.0   
4431  9995546   52  133.5   83.0         0     0.0    1  58.0   NaN   140.0   
4432  9998212   40  141.0   98.0         0     0.0    0  46.0  52.0   126.0   
4433  9999312   39  133.0   86.0         1     0.0    0  46.0  50.0   138.0   

      SYSBP3  DIABP2  DIABP3  CURSMOKE2  CUR

In [24]:
# MICE1
# MICE is going to fill in the nan value by chain equation
import numpy as np
from sklearn.impute import SimpleImputer
# we get the strategy of mean
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X = df_mice.to_numpy()
imp.fit(X)
SimpleImputer()
X = imp.transform(X)
df_micemean = pd.DataFrame(X, columns = ["RANDID", "AGE", "SYSBP", "DIABP", "CURSMOKE", "BPMEDS", "CVD",
                                "AGE2", "AGE3","SYSBP2", "SYSBP3", "DIABP2", "DIABP3", "CURSMOKE2", "CURSMOKE3"])
# print(df_micemean)
df_micemean.to_csv('../data/data_micemean.csv', index=False)
print("", df.shape[0], "rows")

 11627 rows


In [25]:
# MICE2
# MICE is going to fill in the nan value by chain equation
import numpy as np
from sklearn.impute import SimpleImputer
# we use the strategy of getting the most frequency
imp = SimpleImputer(strategy="most_frequent")
X = df_mice.to_numpy()
imp.fit(X)
SimpleImputer()
X = imp.fit_transform(df_mice)
df_micefrequency = pd.DataFrame(X, columns = ["RANDID", "AGE", "SYSBP", "DIABP", "CURSMOKE", "BPMEDS", "CVD",
                                "AGE2", "AGE3","SYSBP2", "SYSBP3", "DIABP2", "DIABP3", "CURSMOKE2", "CURSMOKE3"])
df_micefrequency.to_csv('../data/data_micefrequency.csv', index=False)
print("", df.shape[0], "rows")


 11627 rows
